# Homework 2 

- Implement Naive Bayes model (remember about smoothing). 
- Find a reasonably interesting but not to complicated dataset for which you will be able to use this model to perform binary classification. Do the latter.
- Produce the confussion matrix, calculate accuracy, precission, recall
- Check how your model does against its version from sklearn and logistic regression from sklearn. 

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
twitter_data=pd.read_csv("data\Task_02_Kiryieuski_gen-cl-tw.csv", encoding="ansi")
print(twitter_data.columns)
print(twitter_data['gender'].unique())
print(len(twitter_data['gender']))
twitter_data.head(2)

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')
['male' 'female' 'brand' 'unknown' nan]
20050


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,‰ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)


In [3]:
twitter_data.drop(columns=['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'], inplace=True)
twitter_data.head(3)

,gender,text
0,male,Robbie E Responds To Critics After Win Against...
1,male,‰ÛÏIt felt like they were my friends and I was...
2,male,i absolutely adore when louis starts the songs...


In [4]:
binary_df=twitter_data[(twitter_data['gender']=='female') | (twitter_data['gender']=='male')]
binary_df['num_genders']=binary_df.gender.map({'female':0, 'male':1})
binary_df.reset_index(drop=True, inplace=True)

def remove_punctuation(text):
    import string
    table = str.maketrans(dict.fromkeys(string.punctuation))
    return text.translate(table)

def removeUnASCII(text):
    return text.encode("ascii", errors="ignore").decode()
binary_df['text_w_ascii'] = binary_df['text'].apply(removeUnASCII)
binary_df['witht_punctuation_text'] = binary_df['text_w_ascii'].apply(remove_punctuation)

print( "Unique genders -> ", binary_df['gender'].unique() )
print( "Males -> ", len(binary_df[binary_df['gender']=='male']) )
print( "Females -> ", len(binary_df[binary_df['gender']=='female']) )
print( "New size of DF -> ", len(binary_df))
binary_df.head(10)

Unique genders ->  ['male' 'female']
Males ->  6194
Females ->  6700
New size of DF ->  12894


C:\MyFiles\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\MyFiles\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\MyFiles\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

,gender,text,num_genders,text_w_ascii,witht_punctuation_text
0,male,Robbie E Responds To Critics After Win Against...,1,Robbie E Responds To Critics After Win Against...,Robbie E Responds To Critics After Win Against...
1,male,‰ÛÏIt felt like they were my friends and I was...,1,It felt like they were my friends and I was li...,It felt like they were my friends and I was li...
2,male,i absolutely adore when louis starts the songs...,1,i absolutely adore when louis starts the songs...,i absolutely adore when louis starts the songs...
3,male,Hi @JordanSpieth - Looking at the url - do you...,1,Hi @JordanSpieth - Looking at the url - do you...,Hi JordanSpieth Looking at the url do you us...
4,female,Watching Neighbours on Sky+ catching up with t...,0,Watching Neighbours on Sky+ catching up with t...,Watching Neighbours on Sky catching up with th...
5,female,"Ive seen people on the train with lamps, chair...",0,"Ive seen people on the train with lamps, chair...",Ive seen people on the train with lamps chairs...
6,male,Gala Bingo clubs bought for å£241m: The UK's l...,1,Gala Bingo clubs bought for 241m: The UK's lar...,Gala Bingo clubs bought for 241m The UKs large...
7,female,@_Aphmau_ the pic defines all mcd fangirls/fan...,0,@_Aphmau_ the pic defines all mcd fangirls/fan...,Aphmau the pic defines all mcd fangirlsfanboys...
8,female,@Evielady just how lovely is the tree this yea...,0,@Evielady just how lovely is the tree this yea...,Evielady just how lovely is the tree this year...
9,female,Just put my ass on the line for you and this i...,0,Just put my ass on the line for you and this i...,Just put my ass on the line for you and this i...


In [5]:
from sklearn.model_selection import train_test_split
X_train_tw, X_test_tw, y_train_tw, y_test_tw = train_test_split(binary_df["witht_punctuation_text"],binary_df["num_genders"],
                                                    test_size = 0.1, random_state = 10)
print(len(X_train_tw), len(X_test_tw), len(y_train_tw), len(y_test_tw))

11604 1290 11604 1290


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train_tw)

X_train_vec_tw = vect.transform(X_train_tw)
X_test_vec_tw = vect.transform(X_test_tw)
print('X_train_vec type -> ', type(X_train_vec_tw))
print("Number of features -> ", len(vect.get_feature_names()))
print(vect.get_feature_names()[:10])
print(vect.get_feature_names()[2500:2510])
print(vect.get_feature_names()[5780: 5800])
print(X_train_vec_tw.shape)

X_train_vec type ->  <class 'scipy.sparse.csr.csr_matrix'>
Number of features ->  28196
['00', '000000000000000002344', '007', '00sev', '0105', '0105pm', '012011', '017', '03', '0433355207']
['bdib', 'bdizzz', 'bdo', 'be', 'bea4palestine', 'beach', 'beachamp', 'beachballdelay', 'beachbody', 'beaches']
['danwinchester6', 'dap', 'daphnekiko', 'daquan', 'darabont', 'daraedits', 'dare', 'darealdeegottii', 'dares', 'daria', 'darianmalone35', 'daring', 'dariusphilson2', 'dark', 'darkact', 'darker', 'darkness', 'darkskin', 'darla', 'darling']
(11604, 28196)


In [7]:
class NavieBayesIMPL:
    #MALE=1,FEMALE=0
    def __init__(
        self,
        alpha: float = 0.001,
        a: float = 1.0,
        prob_of_classes = {},
        feature_probs_per_class = {}
    ):
        self.alpha = alpha
        self.a = a
        self.prob_of_classes = prob_of_classes
        self.feature_probs_per_class = feature_probs_per_class
    
    def fit(self, X, y):
        self.prob_of_classes = self._extract_clasess_prob(y)
        self.feature_probs_per_class = self._extract_features_probs(X, y)
        
    def _extract_clasess_prob(self, y):
        prob_of_class = {}
        classes =  np.unique( y.values )
        y_as_list = list(y.values)
        for class_name in classes:
            class_count = y_as_list.count(class_name) 
            prob_of_class[class_name] = class_count/y.size
        return prob_of_class
    
    def _extract_features_probs(self, X, y):
        classes = np.unique(y)
        train_dict_per_class = {}
        
        for clazz in classes:
            clazz_idx = y[y.values==clazz]
            mask = np.in1d(y, clazz_idx)
            sum_per_col = sum(X.toarray()[mask,:]) + self.alpha
            total_words_sum_per_class = sum(sum_per_col)
            train_dict_per_class[clazz] = [word_tot_in_col/(total_words_sum_per_class + len(sum_per_col)+1) for word_tot_in_col in sum_per_col]
        return train_dict_per_class
    
    def predict(self, X):
        label_per_text = []
        prediction = []
        data_len = X.shape[0]
        features_len = X.shape[1]
        data_to_arr=X.toarray()
        for text_idx in range(data_len):
            score_dict = {}
            data_arr_per_text = data_to_arr[text_idx]
            for clazz in self.feature_probs_per_class.keys():
                score_dict[clazz] = 1
                
                feature_prob_per_class = pd.Series( self.feature_probs_per_class[clazz] )
                
                powers = pd.Series( [1+data_arr_per_text[feature_idx] for feature_idx in range(features_len)] ).apply(np.log)
                score_dict[clazz] = sum([power*np.log(probability) for power, probability in zip(powers, feature_prob_per_class)])
                score_dict[clazz] += np.log(self.prob_of_classes[clazz])
            max_score = max(score_dict, key=score_dict.get)
            prediction.append(max_score)
        return prediction

In [8]:
NB = NavieBayesIMPL()
NB.fit(X_train_vec_tw, y_train_tw)

In [9]:
y_test_pred = NB.predict(X_test_vec_tw)

In [10]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_vec_tw, y_train_tw)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [11]:
y_test_pred_sclr_multinom = model.predict(X_test_vec_tw)

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vec_tw, y_train_tw)

C:\MyFiles\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
y_test_pred_log_reg = model.predict(X_test_vec_tw)

In [14]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score, recall_score


conf_mat = confusion_matrix(y_test_tw, y_test_pred)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

conf_mat_sclr_multinom = confusion_matrix(y_test_tw, y_test_pred_sclr_multinom)
conf_mat_normalized_sclr_multinom = conf_mat.astype('float') / conf_mat_sclr_multinom.sum(axis=1)[:, np.newaxis]

conf_mat_log_reg = confusion_matrix(y_test_tw, y_test_pred_log_reg)
conf_mat_normalized_log_reg = conf_mat.astype('float') / conf_mat_log_reg.sum(axis=1)[:, np.newaxis]


In [15]:
print('My MultinomialNB confusion matrix:')
print( conf_mat )
print('Sclearn MultinomialNB confusion matrix:')
print( conf_mat_sclr_multinom )
print('Sclearn LogisticRegression confusion matrix:')
print( conf_mat_log_reg )
print('________________________________________________________________________')
print('My MultinomialNB accuracy -> ', accuracy_score(y_test_tw, y_test_pred) )
print('Sklearn MultinomialNB accuracy -> ', accuracy_score(y_test_tw, y_test_pred_sclr_multinom) )
print('Sklearn LogisticRegression accuracy -> ', accuracy_score(y_test_tw, y_test_pred_log_reg) )
print('________________________________________________________________________')
print('My MultinomialNB precision -> ', precision_score(y_test_tw, y_test_pred) )
print('Sklearn MultinomialNB precision -> ', precision_score(y_test_tw, y_test_pred_sclr_multinom) )
print('Sklearn LogisticRegression precision -> ', precision_score(y_test_tw, y_test_pred_log_reg) )
print('________________________________________________________________________')
print('My MultinomialNB recall -> ', recall_score(y_test_tw, y_test_pred) )
print('Sklearn MultinomialNB recall -> ', recall_score(y_test_tw, y_test_pred_sclr_multinom) )
print('Sklearn LogisticRegression recall -> ', recall_score(y_test_tw, y_test_pred_log_reg) )


My MultinomialNB confusion matrix:
[[438 205]
 [309 338]]
Sclearn MultinomialNB confusion matrix:
[[485 158]
 [339 308]]
Sclearn LogisticRegression confusion matrix:
[[429 214]
 [283 364]]
________________________________________________________________________
My MultinomialNB accuracy ->  0.6015503875968993
Sklearn MultinomialNB accuracy ->  0.6147286821705427
Sklearn LogisticRegression accuracy ->  0.6147286821705427
________________________________________________________________________
My MultinomialNB precision ->  0.6224677716390423
Sklearn MultinomialNB precision ->  0.6609442060085837
Sklearn LogisticRegression precision ->  0.629757785467128
________________________________________________________________________
My MultinomialNB recall ->  0.5224111282843895
Sklearn MultinomialNB recall ->  0.4760432766615147
Sklearn LogisticRegression recall ->  0.5625965996908809
